In [1]:
# Import GaiaXPy Photometric Generator 
from gaiaxpy import generate, PhotometricSystem, plot_spectra
from astropy.io import fits
from astropy.io import votable
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.vizier import Vizier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.ipac.irsa import Irsa
from astropy.table import hstack
from astroquery.utils.tap.core import TapPlus

pd.set_option('display.float_format', '{:.14f}'.format)
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows

def process_line(line):
    # Trim leading and trailing whitespaces, and replace multiple spaces with single space
    cleaned_line = ' '.join(line.strip().split())
    # Replace space with comma
    csv_line = cleaned_line.replace(' ', ',')
    return csv_line

def convert_to_csv(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # Process each line
            csv_line = process_line(line)
            # Write processed line to output file
            f_out.write(csv_line + '\n')

def get_binocs_cluster_ids(cluster_names):
    binocs_mems = Table.read("/scratch/binocs/binocs_cluster_mems.fits")
    gaiaDR3_ids = []

    for row in binocs_mems:
        if row["Name"] in cluster_names:
            gaiaDR3_ids.append(row["GaiaDR3"])

    gaiaSet = set(gaiaDR3_ids)
    return gaiaSet


def gaia_query_binocs_ids(gaiaSet):
    phot_system_list = [PhotometricSystem.SDSS, PhotometricSystem.JKC]
    #query_input = f"select source_id from gaiadr3.gaia_source where source_id in ({gaiaDR3_set_str})"
    synthetic_photometry = generate(list(gaiaSet), photometric_system=phot_system_list, save_file=False)

    return synthetic_photometry

def parse_2mass_string(twomass_string):
    # Extract right ascension and declination from the two-mass string
    delim = ""
    if "+" in twomass_string:
        delim = "+"
    elif "-" in twomass_string:
        delim = "-"

    left, right = twomass_string.split(delim)

    ra_string = left[2:]
    dec_string = right
    ra_hr = ra_string[0:2] + "h"
    ra_min = ra_string[2:4] + "m"
    ra_sec = ra_string[4:6] + "." + ra_string[6:8] + "s"
    ra_2mass = ra_hr + ra_min + ra_sec
    dec_deg = delim + dec_string[0:2] + "d"
    dec_arcmin = dec_string[2:4] + "m"
    dec_arcsec = dec_string[4:6] + "." + dec_string[6:] + "s"
    dec_2mass = dec_deg + dec_arcmin + dec_arcsec
    return ra_2mass, dec_2mass

def find_source_id_based_on_2Mass(twomass_string):
    # Parse the two-mass string into right ascension and declination
    ra_2mass, dec_2mass = parse_2mass_string(twomass_string)
    
    # Query Gaia catalog for the source ID directly
    gaia_query = Vizier(columns=['Source'], catalog='I/355/gaiadr3')
    gaia_result = gaia_query.query_region(SkyCoord(ra=ra_2mass, dec=dec_2mass, frame='icrs'), radius=0.5 * u.arcsecond)
    
    # Process query result
    if gaia_result is not None and len(gaia_result) > 0:
        # Extract the source ID directly from the query result
        source_id = gaia_result[0]['Source'][0]  # Assuming there's only one result
        return source_id
    else:
        return 0  # Return None if no matching sources found

def cross_match_gaia_to_2mass(gaia_source_ids):
    gaia_coords = []
    for id in gaia_source_ids:
        source_id = id
        # Query Gaia database for coordinates of given source ID
        query = "SELECT ra, dec FROM gaiadr3.gaia_source WHERE source_id = {}".format(source_id)
        job = Gaia.launch_job(query)
        gaia_result = job.get_results()
        if len(gaia_result) > 0:
            gaia_coords.append(SkyCoord(ra=gaia_result['ra'][0], dec=gaia_result['dec'][0], unit=(u.degree, u.degree)))
    
    # Cross-match Gaia coordinates with 2MASS catalog using Vizier
    vizier = Vizier(columns=['RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag'])
    vizier.ROW_LIMIT = -1  # Retrieve all rows
    result = vizier.query_region(gaia_coords, radius=5*u.arcsec, catalog='II/246')  # 2MASS catalog
    
    # Combine Gaia and 2MASS results
    if result is not None and len(result) > 0:
        result = result[0]  # Get first table (2MASS catalog)
        cross_matched_table = hstack([gaia_result, result])
        return cross_matched_table
    else:
        return None

def get_real_photometry(cluster):
    input_file = './' + cluster + '.332.txt'  # Change this to your input file name
    output_file = cluster + '332.csv'  # Change this to your output file name

    convert_to_csv(input_file, output_file)
    #colnames = ["2Mass Name",      "ra",      "dec",     'U', 'U_err', 'B', 'B_err', 'V',  'V_err', 'R', 'R_err', 'I', 'I_err', 'SU', 'SU_err', 'SG', 'SG_err', 'SR', 'SR_err', 'SI', 'SI_err', 'SZ', 'SZ_err', 'J', 'J_err', 'H', 'H_err', 'K', 'K_err', 'B1', 'B1_err', 'B2', 'B2_err', 'B3', 'B3_err', 'B4', 'B4_err']
    #               0               1           2           3      4    5   6           7   8           9   10      11      12      13  14      15      16          17      18     19       20      21      22        
    #column_names = ["2Mass Name", "ra", "dec", 'U', 'B', 'V', 'R', 'I', 'SU', 'SG', 'SR', 'SI', 'SZ', 'J', 'H', 'K', 'B1', 'B2', 'B3', 'B4']
    short_col_names = ["2Mass Name", "ra", "dec", "SU", "SG", "SR", "SI", "SZ"]
    cluster_df = pd.read_csv(output_file, header=None, names=short_col_names, usecols=[0, 1, 2,13, 15, 17, 19, 21])
    return cluster_df

def merge_real_synthetic_photometry(cluster_df ,synthetic_photometry): 
    cluster_df['source_id'] = 0
    for index, row in cluster_df.iterrows():
        source_id = find_source_id_based_on_2Mass(row['2Mass Name'])
        cluster_df.at[index, 'source_id'] = int(source_id)

    merged_df = pd.merge(synthetic_photometry, cluster_df, on='source_id', how='inner')

    return merged_df


def plot_merged_real_synthetic_delta(merged_df):
    plt.figure(figsize=(10, 6))
    plt.scatter(merged_df['SG'], merged_df["Sdss_mag_u"]-merged_df['SU'], label='Delta u', marker='.', s=5)
    plt.scatter(merged_df['SG'], merged_df["Sdss_mag_g"]-merged_df['SG'], label='Delta g', marker='.', s=5)
    plt.scatter(merged_df['SG'], merged_df["Sdss_mag_r"]-merged_df['SR'], label='Delta r', marker='.', s=5)
    plt.scatter(merged_df['SG'], merged_df["Sdss_mag_i"]-merged_df['SI'], label='Delta i', marker='.', s=5)
    plt.scatter(merged_df['SG'], merged_df["Sdss_mag_z"]-merged_df['SZ'], label='Delta z', marker='.', s=5)
    plt.ylim(-0.5,0.5)
    plt.xlim(11,20)

    plt.title('Delta Plot for Synthetic vs Real Slone ugriz Magnitudes')
    plt.xlabel('Index')
    plt.ylabel('Magnitude Difference')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    plt.figure(figsize=(10, 6))
    plt.scatter(merged_df["SG"]-merged_df['SI'], merged_df['SG'], label='', marker='.')
    plt.xlim(0,2)
    plt.ylim(20,11)
    plt.show()
        

def create_delta_df(merged_df):
    delta_su = merged_df["Sdss_mag_u"] - merged_df['SU']
    delta_sg = merged_df["Sdss_mag_g"] - merged_df['SG']
    delta_sr = merged_df["Sdss_mag_r"] - merged_df['SR']
    delta_si = merged_df["Sdss_mag_i"] - merged_df['SI']
    delta_sz = merged_df["Sdss_mag_z"] - merged_df['SZ']

    delta_df = pd.DataFrame({
        'delta_su': delta_su,
        'delta_sg': delta_sg,
        'delta_sr': delta_sr,
        'delta_si': delta_si,
        'delta_sz': delta_sz
    })

    return delta_df

def plot_distribution_with_stats(dataframe):
    columns = dataframe.columns
    
    # Plotting
    plt.figure(figsize=(12, 8))
    for column in columns:
        data = dataframe[column]
        mean = np.mean(data)
        std_dev = np.std(data)
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev
        
        # Adjusting bin range
        bin_range = np.linspace(-4, 4, 150)
        
        plt.hist(data, bins=bin_range, alpha=0.5, label=f'{column} (mean={mean:.2f}, std={std_dev:.2f})')
    
    plt.title('Distribution of Columns with Mean and Standard Deviation')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


def remove_x_sigma_outliers_by_column(data, sigma_cnt=3):
    mean = np.mean(data)
    std_dev = np.std(data)
    lower_bound = mean - sigma_cnt * std_dev
    upper_bound = mean + sigma_cnt * std_dev
    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    return filtered_data

def remove_outliers_on_df(delta_df, sigma_cnt=3, iterations=1):
    for i in range(iterations):
        filtered_df = delta_df.apply(remove_x_sigma_outliers_by_column, sigma_cnt=sigma_cnt)
    return filtered_df

def get_irsa_wise_data_2mass(twomass_string, radius=0.5*u.arcsec):
    """
    Retrieve J, H, K and b1-b4 band data from IPAC given a 2MASS ID.
    
    Parameters:
        2mass_id (str): The 2MASS ID.
        
    Returns:
        j_data (astropy.table.Table): J band data.
        h_data (astropy.table.Table): H band data.
        k_data (astropy.table.Table): K band data.
    """
    ra_2mass, dec_2mass = parse_2mass_string(str(twomass_string))
    coords = SkyCoord(ra=ra_2mass, dec=dec_2mass, frame='icrs')
    columns="designation,ra,dec,j_m_2mass,j_msig_2mass,h_m_2mass,h_msig_2mass,k_m_2mass, k_msig_2mass,w1mag,w1sigm,w2mag,w2sigm,w3mag,w3sigm,w4mag,w4sigm"
    # Query all bands
    wise_data = Irsa.query_region(coords, catalog="allsky_4band_p3as_psd", spatial="Cone", radius=radius, columns=columns) 
    jhk_b_bands_row_df = pd.DataFrame({
        "2Mass Name": twomass_string[2:],
        "Wise_Id": wise_data["designation"],
        "ra" : wise_data["ra"],
        "dec" : wise_data["dec"],
        "J_mag" : wise_data['j_m_2mass'], 
        "J_err" : wise_data['j_msig_2mass'], 
        "H_mag" : wise_data['h_m_2mass'], 
        "H_err" : wise_data['h_msig_2mass'],
        'K_mag' : wise_data['k_m_2mass'],
        'K_err' : wise_data['k_msig_2mass'],
        'B1_mag' : wise_data['w1mag'],
        'B1_err' : wise_data['w1sigm'],
        'B2_mag' : wise_data['w2mag'],
        'B2_err' : wise_data['w2sigm'],
        'B3_mag' : wise_data['w3mag'],
        'B3_err' : wise_data['w3sigm'],
        'B4_mag' : wise_data['w4mag'],
        'B4_err' : wise_data['w4sigm'],
    })

    return jhk_b_bands_row_df

def calculate_mag_errors_synthetic(synthetic_photometry):
    synthetic_photometry["Sdss_mag_u_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Sdss_flux_error_u'] / synthetic_photometry['Sdss_flux_u'])
    synthetic_photometry["Sdss_mag_g_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Sdss_flux_error_g'] / synthetic_photometry['Sdss_flux_g'])
    synthetic_photometry["Sdss_mag_r_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Sdss_flux_error_r'] / synthetic_photometry['Sdss_flux_r'])
    synthetic_photometry["Sdss_mag_i_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Sdss_flux_error_i'] / synthetic_photometry['Sdss_flux_i'])
    synthetic_photometry["Sdss_mag_z_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Sdss_flux_error_z'] / synthetic_photometry['Sdss_flux_z'])
    synthetic_photometry["Jkc_mag_U_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Jkc_flux_error_U'] / synthetic_photometry['Jkc_flux_U'])
    synthetic_photometry["Jkc_mag_B_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Jkc_flux_error_B'] / synthetic_photometry['Jkc_flux_B'])
    synthetic_photometry["Jkc_mag_V_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Jkc_flux_error_V'] / synthetic_photometry['Jkc_flux_V'])
    synthetic_photometry["Jkc_mag_R_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Jkc_flux_error_R'] / synthetic_photometry['Jkc_flux_R'])
    synthetic_photometry["Jkc_mag_I_err"] = (2.5 / np.log(10)) * (synthetic_photometry['Jkc_flux_error_I'] / synthetic_photometry['Jkc_flux_I'])


def irsa_query_cluster_by_name(cluster_name, radius=5*u.arcmin):
    columns="designation, ra,dec,j_m_2mass,j_msig_2mass,h_m_2mass,h_msig_2mass,k_m_2mass, k_msig_2mass,w1mag,w1sigm,w2mag,w2sigm,w3mag,w3sigm,w4mag,w4sigm"
    irsa_data = Irsa.query_region(cluster_name, catalog="allsky_4band_p3as_psd", spatial="Cone", radius=radius, columns=columns) 
    return irsa_data

def get_gaia_source_id_from_2mass_fast(twomass_list):
    if twomass_list[0][1]== "M" or twomass_list[0][1]== "D":
        twomass_query_strs = ','.join(["'{}'".format(name) for name in twomass_list.str[2:]])
    else:
        twomass_query_strs = ','.join(["'{}'".format(name) for name in twomass_list])
        
    
    adql_query = """
    SELECT a.source_id, a.original_ext_source_id AS "2Mass_Name", b.original_ext_source_id as "Wise_Id"
    FROM gaiadr3.tmass_psc_xsc_best_neighbour AS a
    JOIN gaiadr3.allwise_best_neighbour AS b ON a.source_id = b.source_id
    WHERE a.original_ext_source_id IN ({})""".format(twomass_query_strs)

    # Run the query
    job = Gaia.launch_job(adql_query)
    result = job.get_results()
    votable_file = votable.from_table(result)
    table = votable_file.get_first_table().to_table()
    gaia_2mass_df = table.to_pandas()
    gaia_2mass_df.rename(columns={'_2Mass_Name': '2Mass Name'}, inplace=True)
    gaia_2mass_df.rename(columns={'_Wise_Id': 'Wise_Id'}, inplace=True)

    return gaia_2mass_df

def get_jhk_only_by_2mass(twomass_list):
    twomass_query_strs = ','.join(["'{}'".format(name) for name in twomass_list.str[2:]])

    query = """
    SELECT *
    FROM fp_psc
    WHERE designation IN ({})
    """.format(twomass_query_strs)
    print(query)
    results = Irsa.query_tap(query).to_table()
    jhk_bands_df = pd.DataFrame({
        "2Mass Name": results["designation"],
        "ra" : results["ra"],
        "dec" : results["dec"],
        "J_mag" : wise_data['j_m'], 
        "J_err" : wise_data['j_msigcom'], 
        "H_mag" : wise_data['h_m'], 
        "H_err" : wise_data['h_msigcom'],
        'K_mag' : wise_data['k_m'],
        'K_err' : wise_data['k_msigcom'],
    })

    return jhk_b_bands_df




# this method took 55 mins


In [6]:
def main():
    #cluster_names = ["NGC_188", "NGC_1960", "NGC_2099", "NGC_2682", "NGC_2158", "NGC_2168", "NGC_2420", "NGC_6791"]
    # cluster_names =["NGC_2682"]
    # gaiaSet = get_binocs_cluster_ids(cluster_names)
    # synthetic_photometry = gaia_query_binocs_ids(gaiaSet)
    # print(synthetic_photometry)
    
    cluster_df = get_real_photometry('NGC2682')
    gaia_2mass_df = get_gaia_source_id_from_2mass_fast(cluster_df["2Mass Name"])
    gaia_2mass_df["source_id"] = gaia_2mass_df["source_id"].astype(int)
    synthetic_photometry = gaia_query_binocs_ids(gaia_2mass_df["source_id"])
    calculate_mag_errors_synthetic(synthetic_photometry=synthetic_photometry)
    merged_df = pd.merge(synthetic_photometry, gaia_2mass_df, on='source_id', how='outer')
    
    
    # #print(get_irsa_wise_data_2mass("2M08521208+1201469"))
    # two_mass_id = "2M08521208+1201469"
    # jhk_b_bands_df = get_irsa_wise_data_2mass(two_mass_id, radius=15*u.arcmin)
    # final_df = pd.merge(merged_df, jhk_b_bands_df, on="Wise_Id", how="inner")
    # print(final_df)
    # final_df.to_csv("FastQuery.csv", index=False)
    
    jhk_b_bands_df = pd.DataFrame()
    # # # # sample_ids = ["2M08521181+1159517",  "2M08521208+1201469"]
    for two_mass_id in cluster_df["2Mass Name"]:
    # # # for two_mass_id in sample_ids:
        jhk_b_bands_row_df = get_irsa_wise_data_2mass(two_mass_id)
        jhk_b_bands_df = pd.concat([jhk_b_bands_df, jhk_b_bands_row_df])
    
    merged_df = pd.merge(merged_df, jhk_b_bands_df, on="2Mass Name", how="outer")
    # # # ids = jhk_b_bands_df["source_id"]
    
    # # # calculate_mag_errors_synthetic(synthetic_photometry=synthetic_photometry)
    # # #print(synthetic_photometry)
    print(merged_df)
    merged_df.to_csv("ngc_ubvri_ugriz_jhkb_all_query.csv", index=False)


    #jhk_b_bands_df.to_csv("jhk_b_bands_03_04.csv", index=False)
    #merged_df = merge_real_synthetic_photometry(cluster_df, synthetic_photometry)
    # plot_merged_real_synthetic_delta(merged_df)
    # delta_df = create_delta_df(merged_df)
    # delta_df = delta_df[(delta_df >= -30) & (delta_df <= 30)]
    # plot_distribution_with_stats(delta_df)
    # filtered_df = remove_outliers_on_df(delta_df, sigma_cnt=2, iterations=10)
    # plot_distribution_with_stats(filtered_df)
    # plot_spectra(synthetic_photometry)
main()

                            source_id        Sdss_mag_u        Sdss_mag_g  \
0   604988410415920768.00000000000000 19.55896437694052 17.55160762692542   
1   604967725853431168.00000000000000 19.26785517067383 17.29858687343150   
2   604967244817098240.00000000000000 12.42319225007301 10.50131852595393   
3   604966866859978240.00000000000000 16.44341877319901 15.12586129315066   
4   604974872679003264.00000000000000 15.71323683853921 14.56261970966727   
5   604912917775108480.00000000000000 17.46008510732634 15.64849364020879   
6   604960647747345792.00000000000000 16.39138214018656 15.19722405921780   
7   604961163143414912.00000000000000 14.94959859399949 13.82028043066560   
8   604967760213166336.00000000000000 17.49395231220826 15.94244839728561   
9   604973635728426752.00000000000000 16.79568520595576 15.39888179492841   
10  604962640612163200.00000000000000 16.03082352310069 14.79967561723802   
11  604912058781650176.00000000000000 15.06655372123685 13.92458741830762   

In [ ]:
cluster_names = ["NGC_188", "NGC_1960", "NGC_2099", "NGC_2682", "NGC_2158", "NGC_2168", "NGC_2420", "NGC_6791"]
cluster_names =["NGC_2682"]
gaiaSet = get_binocs_cluster_ids(cluster_names)
synthetic_photometry = gaia_query_binocs_ids(gaiaSet)
print(synthetic_photometry)
calculate_mag_errors_synthetic(synthetic_photometry=synthetic_photometry)
cluster_df = get_real_photometry('NGC2682')
merged_df = pd.merge(synthetic_photometry, cluster_df, on='source_id', how='outer')
plot_merged_real_synthetic_delta(merged_df)
delta_df = create_delta_df(merged_df)
delta_df = delta_df[(delta_df >= -30) & (delta_df <= 30)]
plot_distribution_with_stats(delta_df)
filtered_df = remove_outliers_on_df(delta_df, sigma_cnt=2, iterations=10)
plot_distribution_with_stats(filtered_df)
plot_spectra(synthetic_photometry)

In [6]:
irsa_data = Irsa.query_region("m67", catalog="fp_psc", spatial="Cone", radius=10*u.arcmin)
twomass_df = pd.DataFrame({
    '2Mass Name': irsa_data['designation'],
    "ra" :  irsa_data["ra"],
    "dec" :  irsa_data["dec"],
    "J_mag" :  irsa_data['j_m'], 
    "J_err" :  irsa_data['j_msigcom'], 
    "H_mag" :  irsa_data['h_m'], 
    "H_err" :  irsa_data['h_msigcom'],
    'K_mag' :  irsa_data['k_m'],
    'K_err' :  irsa_data['k_msigcom']
})
    
gaia_2mass_df = get_gaia_source_id_from_2mass_fast(twomass_df["2Mass Name"])
merged_df = pd.merge(twomass_df, gaia_2mass_df, on="2Mass Name", how="inner")
    
irsa_data = irsa_query_cluster_by_name("m67", 10*u.arcmin)
irsa_df = pd.DataFrame({
    'Wise_Id': irsa_data['designation'],
    # "ra" :  irsa_data["ra"],
    # "dec" :  irsa_data["dec"],
    # "J_mag" :  irsa_data['j_m_2mass'], 
    # "J_err" :  irsa_data['j_msig_2mass'], 
    # "H_mag" :  irsa_data['h_m_2mass'], 
    # "H_err" :  irsa_data['h_msig_2mass'],
    'K_mag' :  irsa_data['k_m_2mass'],
    # 'K_err' :  irsa_data['k_msig_2mass'],
    'B1_mag' :  irsa_data['w1mag'],
    'B1_err' :  irsa_data['w1sigm'],
    'B2_mag' :  irsa_data['w2mag'],
    'B2_err' :  irsa_data['w2sigm'],
    'B3_mag' :  irsa_data['w3mag'],
    'B3_err' :  irsa_data['w3sigm'],
    'B4_mag' : irsa_data['w4mag'],
    'B4_err' :  irsa_data['w4sigm'],
})
        
merged_df = pd.merge(merged_df, irsa_df, on="K_mag", how="inner")
synthetic = gaia_query_binocs_ids(merged_df["source_id"])
calculate_mag_errors_synthetic(synthetic_photometry=synthetic)
final_df = pd.merge(synthetic, merged_df, on="source_id", how="inner")
print(final_df)
print(final_df.columns)

# this method took 16s with cache

                                                                388 [00:00<?, ?spec/s]

              source_id        Sdss_mag_u        Sdss_mag_g        Sdss_mag_r  \
0    604924015970612992 16.09495511443662 14.85500920298744 14.21973580832536   
1    604924015970612992 16.09495511443662 14.85500920298744 14.21973580832536   
2    604924290848519168 14.58183058716333 12.78876595632968 11.99915122449143   
3    604924329503990784               NaN 19.69801638172962 17.98547003868162   
4    604923745388235136 18.27403162283500 16.47444834513815 15.51079159022158   
5    604923985906414080 18.56385656181225 17.31774214015017 16.86174036184271   
6    604923607949288064 15.36645217512017 14.23206014407415 13.81663831173784   
7    604923607949288064 15.36645217512017 14.23206014407415 13.81663831173784   
8    604924393927734528 14.51295202927368 13.13859856171126 12.57703157850651   
9    604924428292443776 18.42110899936150 17.12158143299867 16.59262113888075   
10   604924260784323456 17.91716267140118 16.93770204662209 16.53182059314101   
11   604924260784323456 17.9

In [2]:

cluster_df = get_real_photometry('NGC2682')
twomass_strs = combined_string = ','.join(["'{}'".format(name) for name in cluster_df["2Mass Name"].str[2:]])
print(len(cluster_df["2Mass Name"]))

adql_query = """
SELECT source_id 
FROM gaiadr3.tmass_psc_xsc_best_neighbour
WHERE original_ext_source_id IN ({});""".format(twomass_strs)
print(adql_query)

# Run the query
job = Gaia.launch_job(adql_query)
result = job.get_results()

# # Convert the results to a pandas DataFrame
df = pd.DataFrame(result)
print(df)

1148

SELECT source_id 
FROM gaiadr3.tmass_psc_xsc_best_neighbour
WHERE original_ext_source_id IN ('08521208+1201469','08521181+1159517','08521269+1202592','08501906+1202373','08520330+1158046','08513806+1201243','08512410+1201305','08511298+1157009','08513710+1154599','08520153+1201036','08513190+1153328','08512239+1200182','08513284+1153447','08520041+1156070','08511826+1150196','08512526+1148135','08511710+1148160','08505902+1148291','08511840+1141041','08512003+1151016','08512804+1142347','08505320+1140071','08521641+1152402','08511564+1150561','08511901+1150057','08501827+1155212','08513673+1149590','08514234+1150076','08510714+1135486','08504802+1149414','08511242+1147061','08511773+1146546','08515852+1146529','08513365+1138336','08513257+1148240','08511904+1140156','08505336+1148573','08514465+1141510','08511269+1152423','08512988+1150228','08511176+1150018','08512436+1149504','08512214+1148278','08505430+1155152','08511628+1144328','08511774+1150055','08505653+1138081','0851152

ValueError: DataFrame constructor not properly called!

In [4]:
from astroquery.ipac.irsa import Irsa

# Irsa.list_catalogs()
# wise_data = Irsa.query_region("m67", catalog="allsky_4band_p3as_psd", spatial="Cone", radius=1 * u.arcmin)
# print(wise_data.columns)
# print(wise_data)
#fp_psc, allsky_4band_p3as_psd
cluster_df = get_real_photometry("NGC2682")
  
query = """
SELECT ra, dec, designation
FROM fp_psc
WHERE designation = '08521208+1201469'
"""
results = Irsa.query_tap(query).to_table()
print(results)
    
query2 = """
SELECT ra, dec, designation
FROM allsky_4band_p3as_psd
WHERE dec = 12.029708 AND ra = 133.050342
"""
                
results2 = Irsa.query_tap(query2).to_table()
print(results2)

    ra        dec       designation   
   deg        deg                     
---------- ---------- ----------------
133.050342  12.029708 08521208+1201469
 ra dec designation
deg deg            
--- --- -----------


In [24]:
from astroquery.ipac.irsa import Irsa

cluster_df = get_real_photometry("NGC2682")
query = """
SELECT *
FROM fp_psc
WHERE designation = '08521208+1201469'
"""
results = Irsa.query_tap(query).to_table()
print(results)

for id in cluster_df["2Mass Name"]:
    query = """
    SELECT ra, dec, designation
    FROM fp_psc
    WHERE designation = '{}'
    """.format(id[2:])
    results = Irsa.query_tap(query).to_table()
    # print(results)
    ra = results["ra"][0]
    dec = results["dec"][0]
    formatted_ra = "{:.6f}".format(ra)
    formatted_dec = "{:.6f}".format(dec)

    query2 = """
    SELECT ra, dec, designation
    FROM allsky_4band_p3as_psd
    WHERE dec = {} and ra = {}
    """.format(formatted_dec, formatted_ra)       
    print(query2)    
    results2 = Irsa.query_tap(query2).to_table()
    print(results2)

    # coords = SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs')
    # columns="ra,dec,j_m_2mass,j_msig_2mass,h_m_2mass,h_msig_2mass,k_m_2mass, k_msig_2mass,w1mag,w1sigm,w2mag,w2sigm,w3mag,w3sigm,w4mag,w4sigm"
    # # Query all bands
    # wise_data = Irsa.query_region(coords, catalog="allsky_4band_p3as_psd", spatial="Cone", radius=1 * u.arcsec, columns=columns) 
    
    # print(wise_data)
    


    ra        dec     err_maj err_min ... coadd_key coadd        htm20       
   deg        deg      arcsec  arcsec ...                                    
---------- ---------- ------- ------- ... --------- ----- -------------------
133.050342  12.029708    0.11    0.06 ...    174318     9 4804677833771681280

    SELECT ra, dec, designation
    FROM allsky_4band_p3as_psd
    WHERE dec = 12.029708 and ra = 133.050342
    
 ra dec designation
deg deg            
--- --- -----------

    SELECT ra, dec, designation
    FROM allsky_4band_p3as_psd
    WHERE dec = 11.997709 and ra = 133.049246
    
 ra dec designation
deg deg            
--- --- -----------

    SELECT ra, dec, designation
    FROM allsky_4band_p3as_psd
    WHERE dec = 12.049789 and ra = 133.052900
    
 ra dec designation
deg deg            
--- --- -----------

    SELECT ra, dec, designation
    FROM allsky_4band_p3as_psd
    WHERE dec = 12.043704 and ra = 132.579424
    
 ra dec designation
deg deg            
--- --- 

KeyboardInterrupt: 

In [33]:
wise_data = Irsa.query_region("08511325+1157526", catalog="allsky_4band_p3as_psd", spatial="Cone") 
print(wise_data)

NameResolveError: Unable to find coordinates for name '08511325+1157526' using https://cds.unistra.fr/cgi-bin/nph-sesame/A?08511325%2B1157526

In [2]:
cluster_df = get_real_photometry('NGC2682')
gaia_2mass_df = get_gaia_source_id_from_2mass_fast(cluster_df["2Mass Name"])
gaia_2mass_df["source_id"] = gaia_2mass_df["source_id"].astype(int)
# synthetic_photometry = gaia_query_binocs_ids(gaia_2mass_df["source_id"])
gaiaSet = gaia_2mass_df["source_id"]
phot_system_list = [PhotometricSystem.SDSS, PhotometricSystem.JKC]
    #query_input = f"select source_id from gaiadr3.gaia_source where source_id in ({gaiaDR3_set_str})"
synthetic_photometry = generate(list(gaiaSet), photometric_system=phot_system_list, save_file=False)

synthetic_photometry

,source_id,Sdss_mag_u,Sdss_mag_g,Sdss_mag_r,Sdss_mag_i,Sdss_mag_z,Sdss_flux_u,Sdss_flux_g,Sdss_flux_r,Sdss_flux_i,Sdss_flux_z,Sdss_flux_error_u,Sdss_flux_error_g,Sdss_flux_error_r,Sdss_flux_error_i,Sdss_flux_error_z,Jkc_mag_U,Jkc_mag_B,Jkc_mag_V,Jkc_mag_R,Jkc_mag_I,Jkc_flux_U,Jkc_flux_B,Jkc_flux_V,Jkc_flux_R,Jkc_flux_I,Jkc_flux_error_U,Jkc_flux_error_B,Jkc_flux_error_V,Jkc_flux_error_R,Jkc_flux_error_I
0,604988410415920768,19.55896437694052,17.55160762692542,16.14964181811394,15.60673129545878,15.30599490398802,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,18.81631182184422,18.12105441269133,16.72875528192608,15.82788944562234,15.05457667556810,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
1,604967725853431168,19.26785517067383,17.29858687343150,15.92934534598856,15.42230299902287,15.13129485549393,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,18.54471542954021,17.86350969223151,16.49303823298787,15.61779221548932,14.87964033489240,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
2,604967244817098240,12.42319225007301,10.50131852595393,9.63542464656879,9.34462755551134,9.18759974989545,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,11.65967682000372,11.01935074964761,9.95892364560898,9.38152312043223,8.86691575310679,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
3,604966866859978240,16.44341877319901,15.12586129315066,14.59030446793727,14.45186881344140,14.40166197649508,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,15.66899741981397,15.49799970722340,14.78596371504696,14.37957701603641,14.01555178999749,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
4,604974872679003264,15.71323683853921,14.56261970966727,14.13526712045331,14.01461505436267,13.97688373087328,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,14.92642856292757,14.88600031875231,14.28575606679392,13.92940970995650,13.58366355383690,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
5,604912917775108480,17.46008510732634,15.64849364020879,14.79757275423591,14.56206808728401,14.43894755286443,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,16.69963526585160,16.11946828761490,15.13638565400604,14.55943546707341,14.09763481677137,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000
6,604960647747345792,16.39138214018656,15.19722405921780,14.76429534892362,14.65335179972268,14.63053168686591,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,15.60610172490611,15.52296881784788,14.92048563802185,14.56019313876273,1

In [8]:
# fit_table = Table.read("./ngc_ubvri_ugriz_jhkb_all_query.csv--result_table.fits")
# fit_table
fit_table = Table.read("./binocs/testradecfits.fits")
fit_table


source_id,Sdss_mag_u,Sdss_mag_g,Sdss_mag_r,Sdss_mag_i,Sdss_mag_z,Sdss_flux_u,Sdss_flux_g,Sdss_flux_r,Sdss_flux_i,Sdss_flux_z,Sdss_flux_error_u,Sdss_flux_error_g,Sdss_flux_error_r,Sdss_flux_error_i,Sdss_flux_error_z,Jkc_mag_U,Jkc_mag_B,Jkc_mag_V,Jkc_mag_R,Jkc_mag_I,Jkc_flux_U,Jkc_flux_B,Jkc_flux_V,Jkc_flux_R,Jkc_flux_I,Jkc_flux_error_U,Jkc_flux_error_B,Jkc_flux_error_V,Jkc_flux_error_R,Jkc_flux_error_I,Sdss_mag_u_err,Sdss_mag_g_err,Sdss_mag_r_err,Sdss_mag_i_err,Sdss_mag_z_err,Jkc_mag_U_err,Jkc_mag_B_err,Jkc_mag_V_err,Jkc_mag_R_err,Jkc_mag_I_err,2Mass Name,ra,dec,J_mag,J_err,H_mag,H_err,K_mag,K_err,Wise_Id_x,Wise_Id_y,B1_mag,B1_err,B2_mag,B2_err,B3_mag,B3_err,B4_mag,B4_err
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes16,float64,float64,float32,float32,float32,float32,float32,float32,bytes19,bytes19,float32,float32,float32,float32,float32,float32,float32,float32
604917629355039360,16.14427099480823,14.71615592494748,14.029036790765183,13.798941716806802,13.675590090304645,1.2655858254214585e-29,4.7156072857070256e-29,8.879434003681652e-29,1.097547470749078e-28,1.2296003097013817e-28,2.606773007868747e-31,7.942950370813745e-32,1.3955412974952095e-31,9.794335447311241e-32,1.6753294569776527e-31,15.368751112822075,15.122340590882576,14.293082716852986,13.79492471552971,13.332252382966963,2.927248248873618e-17,5.67994020373688e-17,6.993376555181971e-17,6.634834798505074e-17,5.420745860214563e-17,5.90303609854806e-19,1.0696836742519553e-19,1.09361115253194e-19,7.878566060963464e-20,4.18501335520824e-20,0.022363302238209145,0.0018288097943034026,0.0017064034839354017,0.0009688933627111323,0.0014793147268970393,0.02189476076261722,0.0020447297878417067,0.001697853980637392,0.0012892626076980891,0.0008382279181348721,08512327+1148265,132.846987,11.807363,12.714,0.0217,12.238,0.0188,12.133,0.0198,J085123.27+114826.3,J085123.27+114826.4,11.707,0.016,11.641,0.01,11.119,0.224,8.213,--
604917629355042176,12.987132159952104,11.018553629169325,10.128813212622084,9.85300448101345,9.694149005015156,2.318179900538962e-28,1.4209491884858008e-27,3.2245915749807355e-27,4.157174049369754e-27,4.812163758303182e-27,4.05812856385938e-30,1.1690202267730944e-30,2.2726687511390588e-30,1.1217431632678673e-30,2.0348013268210343e-30,12.223561772775248,11.548295099137984,10.460214845585682,9.879012086035878,9.375759490866102,5.303166845590457e-16,1.5274322652846556e-15,2.386899105767896e-15,2.444528841391468e-15,2.0732716236699055e-15,9.159522452902471e-18,1.5805686541350481e-18,1.5592095962102687e-18,1.272710197999589e-18,5.001135751608859e-19,0.019006536569145742,0.0008932392478119851,0.0007652190012773512,0.0002929675665093874,0.0004590985637609478,0.018752616002795097,0.0011235068493072845,0.0007092425085421313,0.0005652735679512299,0.00026190076053905987,08512280+1148016,132.845,11.800464,8.56,0.0227,8.075,0.0325,7.942,0.0241,J085122.79+114801.7,J085122.80+114801.7,7.94,0.005,7.987,0.006,7.918,0.012,8.421,0.529
604917663716360576,11.675521744377072,10.803292244131434,10.474597642356791,10.303893142897458,10.271014025065291,7.758741850903578e-28,1.7325393305055557e-27,2.3450977125438262e-27,2.744370503244546e-27,2.828748847194088e-27,1.64954170859067e-29,2.4065753496989234e-30,7.171870444724185e-30,1.8842870447570455e-30,9.891771056679073e-30,10.891231644409658,11.090158820416288,10.563600716635925,10.251969992157406,9.881981064176358,1.8091198642453166e-15,2.329251971937474e-15,2.1700998320444e-15,1.7338519177149977e-15,1.3006713506638508e-15,3.488739958684298e-17,7.524909005876982e-18,2.8529651063689116e-18,3.850886935069242e-18,8.97625327206553e-19,0.023083216179784347,0.0015081366065635752,0.0033204413428151824,0.000745467371198868,0.003796679926557707,0.02093753629591616,